# LangGraph with Multi-Agent Workflow

Multiple agents interacting in Workflow design

In [1]:
import os
import json
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    ChatMessage,
    FunctionMessage,
    HumanMessage,
)
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import FunctionMessage
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt.tool_executor import ToolExecutor, ToolInvocation
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from typing import TypedDict, Annotated

In [2]:
# OpenAI API Key solely to use embedding model
os.environ["OPENAI_API_KEY"] = ""
os.environ["TAVILY_API_KEY"] = ""

In [3]:
# Tools to be called by LLM
tools = [TavilySearchResults(max_results=1)]

tool_executor = ToolExecutor(tools) # Can invoke an action???

In [4]:
# Load LLM
llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0.1)

In [5]:
# Bind tools to llm
llm_bound = llm.bind_tools(tools)

In [6]:
sample_call = llm_bound.invoke("who played against Universitario last night?")
sample_call


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_f12XooXziw7ecQ2iY75U7CJF', 'function': {'arguments': '{"query":"Universitario match result last night"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 90, 'total_tokens': 114}, 'model_name': 'gpt-4-turbo-preview', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f798cd8a-e3c6-4778-89e6-a5de3717c924-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Universitario match result last night'}, 'id': 'call_f12XooXziw7ecQ2iY75U7CJF'}])

In [10]:
print(sample_call.additional_kwargs)
print(sample_call.additional_kwargs.keys())
sample_call.tool_calls

{'tool_calls': [{'id': 'call_f12XooXziw7ecQ2iY75U7CJF', 'function': {'arguments': '{"query":"Universitario match result last night"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}
dict_keys(['tool_calls'])


[{'name': 'tavily_search_results_json',
  'args': {'query': 'Universitario match result last night'},
  'id': 'call_f12XooXziw7ecQ2iY75U7CJF'}]

In [31]:
# json.loads(last_message.additional_kwargs["function_call"]["arguments"])

print(sample_call.additional_kwargs.keys())
print(sample_call.additional_kwargs["tool_calls"])

# json.loads(sample_call.tool_calls[0]["args"]) # ERROR, single quote string like dict cannot be converted to dict

json.loads(sample_call.additional_kwargs["tool_calls"][0]["function"]["arguments"])



dict_keys(['tool_calls'])
[{'id': 'call_f12XooXziw7ecQ2iY75U7CJF', 'function': {'arguments': '{"query":"Universitario match result last night"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]


{'query': 'Universitario match result last night'}

The tool invocation is the recipe needed to be executed

In [35]:
invocation_sample = ToolInvocation(
        tool=sample_call.additional_kwargs["tool_calls"][0]["function"]["name"],
        tool_input=json.loads(sample_call.additional_kwargs["tool_calls"][0]["function"]["arguments"]),
    )

invocation_sample

ToolInvocation(tool='tavily_search_results_json', tool_input={'query': 'Universitario match result last night'})

The executor (made up of the tools themselves) will execute the recipe (invocation)

In [36]:
tool_executor.invoke(invocation_sample)

[{'url': 'https://www.sofascore.com/team/football/universitario/2305',
  'content': 'Universitario previous match was against Comerciantes Unidos in Liga 1, the match ended with result 6 - 0 (Universitario won the match). Universitario fixtures tab is showing the last 100 football matches with statistics and win/draw/lose icons. There are also all Universitario scheduled matches that they are going to play in the future.'}]

### Build Agent Graph

In [13]:
# Agent state to build graph
class AgentState(TypedDict):
    # The `add_messages` function within the annotation defines
    # *how* updates should be merged into the state.
    messages: Annotated[list, add_messages]

#### Define nodes

In [ ]:
def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]
    # If there is no function call, then we finish
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"

# Node to call model (tool calling, ie. output provides expected structured entry)
def call_model(state):
    messages = state['messages']
    response = llm_bound.invoke(messages)
    
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

# Node to actually execute tool
def call_tool(state):
    messages = state['messages']
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]

    # We construct an ToolInvocation from the function_call
    action = ToolInvocation(
        
        # Get the name of the tool to use
        tool=last_message.additional_kwargs["function_call"]["name"],

        
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"]),
    )
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}